In [161]:
import pandas as pd
import math

In [162]:
from datetime import datetime

def is_minute_multiple_of_five():
    """
    Check if the current minute is a multiple of 5.
    
    :return: True if the current minute is a multiple of 5, otherwise False.
    """
    current_minute = datetime.now().minute
    return current_minute % 5 == 0


In [163]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime
from requests.exceptions import HTTPError

# Constants
jita_station_id = 60003760  # Jita IV - Moon 4 - Caldari Navy Assembly Plant
base_url = "https://esi.evetech.net/latest"

def get_market_orders_jita(page=1):
    """
    Fetch market orders for Jita from ESI API.
    
    :param page: Page number for pagination.
    :return: JSON response containing market orders.
    """
    url = f"{base_url}/markets/10000002/orders/"
    params = {
        "order_type": "sell",
        "page": page,
        "structure_id": jita_station_id
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()  # This will raise an HTTPError for bad responses
    
    return response.json()

def download_all_orders():
    """
    Download all sell orders from Jita.
    
    :return: A list containing all market orders.
    """
    all_orders = []
    page = 1

    while True:
        try:
            orders = get_market_orders_jita(page)
            if not orders:
                break
            all_orders.extend(orders)
            print(f"Fetched page {page}, orders: {len(orders)}")
            page += 1
            time.sleep(1)  # Be polite to the API, limit request rate
        except HTTPError as e:
            if e.response.status_code == 404:
                print(f"Page {page} not found. Ending download.")
                break
            else:
                raise  # Re-raise the exception if it's not a 404 error
    
    return all_orders




In [164]:
raw_data = pd.read_csv('data/eve_ref_market_orders/latest/market-orders-latest.v3.csv')

In [165]:
# Download all orders
# orders = download_all_orders()

# Convert the list of orders to a DataFrame
# raw_data = pd.DataFrame(orders)

# # Get the current date and time
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# # Create the directory if it doesn't exist
# output_dir = f'data/{current_datetime}'
# os.makedirs(output_dir, exist_ok=True)

# # Save the DataFrame to a CSV file in the directory
# output_file = f"{output_dir}/jita_sell_orders.csv"
# # raw_data.to_csv(output_file, index=False)

# print(f"Total orders downloaded: {len(orders)}")
# print(f"Orders saved to {output_file}")


In [166]:
def snipe(orders, type_id, station_id, sell_spread_threshold = 0.5, profit_margin=0, cost_threshold = math.inf):
    # get all orders for type_id
    # orders = data[data['type_id'] == type_id]
    # get the lowest sell order
    snipe_found = False
    trade_dict = {}

    # Filter orders for sell orders
    sell_orders = orders[orders['is_buy_order'] == False].sort_values(by='price', ascending=True)
    buy_orders = orders[orders['is_buy_order'] == True].sort_values(by='price', ascending=False)
    if sell_orders.empty or buy_orders.empty:
        # print(f"No sell orders found for type_id {type_id} at station {station_id}")
        return False  
    else:
        
        
        
        
        lowest_sell_price = sell_orders['price'].min()
        lowest_sell_volume = sell_orders['volume_remain'].iloc[0]
        
        if len(sell_orders) == 1:
            second_lowest_sell_price = lowest_sell_price
        else:
            second_lowest_sell_price = sell_orders['price'].iloc[1]
        
        highest_buy_price = buy_orders['price'].max()

            
        spread = abs(highest_buy_price - lowest_sell_price)
        difference_to_next_sell = abs(second_lowest_sell_price - lowest_sell_price)

        # check if the spread between buy/sell is smaller than the spread between the lowest sell and the second lowest sell
        # if TRUE then we have a snipe - we can buy at the lowest sell price and relist for an easy profit (in theory)
        # This should be more dynamic than simply checking for an exact match between the buy/sell spread 
        margin = second_lowest_sell_price - lowest_sell_price

        profit = margin*lowest_sell_volume
        
        cost = lowest_sell_price*lowest_sell_volume
        
        if spread < (difference_to_next_sell * sell_spread_threshold) and profit > profit_margin and cost < cost_threshold or lowest_sell_price == highest_buy_price and profit > profit_margin and cost < cost_threshold:

            trade_dict['type_id'] = type_id
            trade_dict['station_id'] = station_id
            trade_dict['profit'] = profit
            trade_dict['cost'] = cost
            
            
            
            # volume = orders[orders['is_buy_order'] == False]['volume'].sum()
            print("--------------------------------------------------------------------------")
            print(f"Type ID {type_id} is a snipe at station: {station_id}.") 
            print(f"Profit: {profit} ISK || Cost: {cost} ISK")
            print(f"Lowest sell: {lowest_sell_price} Highest buy: {highest_buy_price} volume: {lowest_sell_volume}") 
            print(f"Second sell: {second_lowest_sell_price} ~ Margin: {margin} ISK")
            print("--------------------------------------------------------------------------")

                        
            return trade_dict
        else:
            return trade_dict

In [167]:
def filter_data(data, station_id, sell_spread_threshold=0.8, profit_margin=0, cost_threshold=math.inf):

    # data = data[data['location_id'] == station_id]
    station_ids = data['location_id'].unique()
    type_ids = data['type_id'].unique()
    # for station_id in station_ids:
    station_data = data[data['location_id'] == station_id]
    snipeFound = False
    
    snipes = []
    
    for type_id in type_ids:
        orders = station_data[station_data['type_id'] == type_id]
        snipe_dict = snipe(orders, type_id, station_id, sell_spread_threshold=sell_spread_threshold, profit_margin=profit_margin, cost_threshold=cost_threshold)
        snipes.append(snipe_dict)
    if not snipes:
            print("          No snipes found. Back to sleep.")
            print("==========================================")
    return snipes

In [168]:
station_ids = {
    'Amarr': 60008494,
    'Jita': 60003760,
    # 'Dodixie': 60011866,
    # 'Hek': 60005686,
    # 'Rens': 60004588
}

In [169]:

for station_name, station_id in station_ids.items():

    print("=====================================")
    print(f"Checking for snipes at hub: {station_name}")
    print("=====================================")
    snipes = filter_data(raw_data, station_id,sell_spread_threshold=0.8, profit_margin = 100000, cost_threshold = 5000000)

Checking for snipes at hub: Jita
--------------------------------------------------------------------------
Type ID 195 is a snipe at station: 60003760.
Profit: 154326.00000000003 ISK || Cost: 272785.0 ISK
Lowest sell: 30.65 Highest buy: 30.51 volume: 8900
Second sell: 47.99 ~ Margin: 17.340000000000003 ISK
--------------------------------------------------------------------------
--------------------------------------------------------------------------
Type ID 197 is a snipe at station: 60003760.
Profit: 863820.0 ISK || Cost: 207000.0 ISK
Lowest sell: 115.0 Highest buy: 60.0 volume: 1800
Second sell: 594.9 ~ Margin: 479.9 ISK
--------------------------------------------------------------------------
--------------------------------------------------------------------------
Type ID 215 is a snipe at station: 60003760.
Profit: 1396335.6000000003 ISK || Cost: 2086040.7599999998 ISK
Lowest sell: 4.93 Highest buy: 4.73 volume: 423132
Second sell: 8.23 ~ Margin: 3.3000000000000007 ISK
----